### Prompt Templates

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai.llms import OpenAI
llm = OpenAI()

In [4]:
from langchain.prompts import PromptTemplate

In [11]:
prompt_template = PromptTemplate.from_template(
	'''
	Responda a seguinte pergunta do usuário:
	{pergunta}
	'''
)
prompt_template

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\n\tResponda a seguinte pergunta do usuário:\n\t{pergunta}\n\t')

In [12]:
print(prompt_template.format(pergunta='O que é um buraco negro?'))


	Responda a seguinte pergunta do usuário:
	O que é um buraco negro?
	


In [ ]:
prompt_template = PromptTemplate.from_template(
	'''
	Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
	{pergunta}
	'''
)
print(prompt_template.format(n_palavras=10, pergunta='O que é um buraco negro?'))


	Responda a seguinte pergunta do usuário em até 10 palavras:
	O que é um buraco negro?
	


In [15]:
# Limitando o número de palavras que o chat vai responder a pergunta do usuário

prompt_template = PromptTemplate.from_template(
	'''
	Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
	{pergunta}
	''', partial_variables={'n_palavras': 20}
)
print(prompt_template.format(pergunta='O que é um buraco negro?'))


	Responda a seguinte pergunta do usuário em até 20 palavras:
	O que é um buraco negro?
	


### Composing prompts - Unindo múltiplos prompts

In [18]:
template_word_count = PromptTemplate.from_template(
	'''
	Responda a pergunta em até {n_palavras} palavras.
	'''
)

template_lingua = PromptTemplate.from_template(
	'''
	Retorne a resposta na {lingua}.
	'''
)

template_final = (
	template_word_count +  template_lingua + 'Responda a pergunta seguinte seguindo as instruções: {pergunta}'
)

print(template_final.template)


	Responda a pergunta em até {n_palavras} palavras.
	
	Retorne a resposta na {lingua}.
	Responda a pergunta seguinte seguindo as instruções: {pergunta}


In [20]:
prompt = template_final.format(n_palavras=10, lingua='Inglês', pergunta='O que é uma estrela?')
llm.invoke(prompt)

'\nA star is a luminous sphere of plasma.'

### Templates para AI Chat

In [22]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template('Essa é a minha dúvida: {duvida}.')
chat_template.format_messages(duvida='Quem foi pitágoras?')


[HumanMessage(content='Essa é a minha dúvida: Quem foi pitágoras?.', additional_kwargs={}, response_metadata={})]

In [25]:
chat_template = ChatPromptTemplate.from_messages(
	[
		('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
		('human', 'Olá, como vai?'),
		('ai', 'Melhor agora!, como posso ajudá-lo?'),
		('human', '{pergunta}')
	]
)

print(chat_template)

input_variables=['nome_assistente', 'pergunta'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], input_types={}, partial_variables={}, template='Você é um assistente engraçado e se chama {nome_assistente}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Olá, como vai?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Melhor agora!, como posso ajudá-lo?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='{pergunta}'), additional_kwargs={})]


In [26]:
chat_template.format(nome_assistente='Dini', pergunta='Quem foi Pitágoras?')

'System: Você é um assistente engraçado e se chama Dini\nHuman: Olá, como vai?\nAI: Melhor agora!, como posso ajudá-lo?\nHuman: Quem foi Pitágoras?'

In [27]:
from langchain_openai.chat_models import ChatOpenAI

chat = OpenAI()

chat.invoke(chat_template.format(nome_assistente='Dini', pergunta='Quem foi Pitágoras?'))

'\nAI: Pitágoras foi um famoso matemático e filósofo grego que viveu no século VI a.C. Ele é conhecido por suas contribuições para a geometria e por sua teoria dos números, incluindo o famoso Teorema de Pitágoras.'

### Templates de Few-Shot prompting para LLMs

In [30]:
exemplos = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

In [32]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
	input_variables=['pergunta', 'resposta'],
	template='Pergunta {pergunta}\n{resposta}'
)

example_prompt.format(**exemplos[0])

'Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'

In [33]:
prompt = FewShotPromptTemplate(
	examples=exemplos,
	example_prompt=example_prompt,
	suffix='Pergunta: {input}',
	input_variables=['input']
)

In [34]:
print(prompt.format(input='Quem fez mais gols, romário ou pelé?'))

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 


Pergunta Quando nasceu o fundador do craigslist?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 


Pergunta Quem foi o avô materno de George Washington?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: 

In [35]:
llm.invoke(prompt.format(input='Quem fez mais gols, romário ou pelé?'))

'\nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Quantos gols Romário fez em sua carreira?\nResposta intermediária: Romário fez 772 gols em sua carreira.\nPergunta de acompanhamento: Quantos gols Pelé fez em sua carreira?\nResposta intermediária: Pelé fez 1281 gols em sua carreira.\nEntão a resposta final é: Pelé'

### Templates de few-shot prompting para chats

In [37]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

In [41]:
example_prompt = ChatPromptTemplate.from_messages(
	[
		('human', '{pergunta}'),
		('ai', '{resposta}')
	]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={})]


In [49]:
few_shot_template = FewShotChatMessagePromptTemplate(
	examples=exemplos,
	example_prompt=example_prompt
)
prompt_template = ChatPromptTemplate.from_messages(
	[
		few_shot_template,
		('human', '{input}')
	]
)

prompt = prompt_template.format_messages(input='Quem fez mais gols, romário ou pelé?')
chat.invoke(prompt)

'\nAI: São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos gols Pelé fez em sua carreira? \nResposta intermediária: Pelé fez 1283 gols em sua carreira. \nPergunta de acompanhamento: Quantos gols Romário fez em sua carreira? \nResposta intermediária: Romário fez 772 gols em sua carreira. \nEntão a resposta final é: Pelé'